In [2]:
import os
import pandas as pd
import sqlite3 as sql

In [45]:
db_base_url = "data/Footbit2_471DFB31-DC3A-4102-899A-55C904FDFF06"
base_extracted_url = "data_extracted"

main_conn = sql.connect(os.path.join(db_base_url, 'main.db'))

transactional_dbs_file = pd.read_sql_query("SELECT * FROM transactional_database", main_conn).iloc[0].file_name
transactional_conn = sql.connect(os.path.join(db_base_url, 'transactional', transactional_dbs_file))

# data_streams = pd.read_sql_query("SELECT * FROM data_stream where device_name == 'footbit'", main_conn).name.to_list()
data_streams = ['ppg']

experiments = pd.read_sql_query("SELECT * FROM experiment", main_conn)
experiments.start = pd.to_datetime(experiments.start)
experiments.end = pd.to_datetime(experiments.end)

In [46]:
data_streams

['accel', 'ppg']

In [40]:
experiments.head(10)

,id,name,uuid,description,created_at,start,end,ts,uploaded,active,track_gps
0,1,exp1,58356CC4-11F8-4957-B09A-407BC8037E29,,2023-02-08 16:37:08.979,2023-02-08 16:37:08.978,2023-02-08 16:37:21.014,2023-02-08 16:37:08.978,0,0,0


In [66]:
# extract all experiments into csvs
for exp in experiments.itertuples():
    # make a dir for this experiment
    exp_url = os.path.join(base_extracted_url, exp.name)
    if not os.path.exists(exp_url):
        os.makedirs(exp_url)
    
    start_date = exp.start
    start_ts = exp.start.timestamp() * 1000000
    end_date = exp.end
    end_ts = exp.end.timestamp() * 1000000
    
    # get configurations for each datastream
    for ds in data_streams:
        sql = f"SELECT * FROM {ds}_config WHERE ts BETWEEN '{start_date}' AND '{end_date}'"
        configs = pd.read_sql_query(sql, main_conn)
        configs = configs.drop(['id', 'uploaded', 'device'], axis=1)
        configs = configs.set_index('ts')
        configs.to_csv(os.path.join(exp_url, f'{ds}_config.csv'))
        
    # save actual data stream data
    for ds in data_streams:
        sql = f"SELECT * FROM {ds} WHERE ts BETWEEN {start_ts} AND {end_ts}"
        print(sql)
        data = pd.read_sql_query(sql, transactional_conn)
        data = data.drop(['device', 'created_at', 'uploaded', 'id'], axis=1)
        data = data.set_index('ts')
        data.to_csv(os.path.join(exp_url, f'{ds}_data.csv'))
        
    # collect heart rate
    sql = f"SELECT * FROM heart_rate WHERE ts BETWEEN {start_ts} AND {end_ts}"
    data = pd.read_sql_query(sql, transactional_conn)
    data = data.drop(['device_name', 'created_at', 'uploaded', 'id'], axis=1)
    data = data.set_index('ts')
    data.to_csv(os.path.join(exp_url, 'heart_rate_data.csv'))

SELECT * FROM accel WHERE ts BETWEEN 1675874228978000.0 AND 1675874241014000.0
SELECT * FROM ppg WHERE ts BETWEEN 1675874228978000.0 AND 1675874241014000.0
